# コンパイラの仕組み

JavaやC/C++のようなコンパイル言語における、コンパイラの役割や仕組みを説明します。

## コンパイラの役割

コンパイラの役割は、プログラムが記述されたソースファイルから、実行環境が認識できる形式（バイトコードやオブジェクトファイル、実行ファイル）へ変換することです。

なぜこのような変換が必要かというと、実行環境では複雑なプログラムを認識できず、シンプルな命令しか処理できないためです。  
このシンプルな命令とは、JavaであればJava-VMが認識できる命令、C/C++であればCPUが認識できる命令となります。

これら命令はざっくり言うと以下のようなものとなります。

- 変数に相当するものは数個程度の固定数（プログラムのように無尽蔵には作れない）
- 少ない変数でメモリとのやり取りを駆使して計算や処理を行う
- forやwhileのようなループは無く、基本的には goto のようなジャンプ命令で代替え
- 関数呼び出しに近い call 命令はある

Java-VMは、Java専用に作られていることから、CPUにはないコンパイル前のJavaプログラムと相性の良い命令（newやキャスト、例外関連など）もありますが、基本的には上記のような少ない命令で実行できるようにコンパイルされます。  
C/C++は、各CPUで動くように、同じプログラムでもCPUが異なれば、別のバイト形式へ変換されます。

## コンパイラの要素

コンパイラは、プログラムからバイト形式への単純な変換ルールで出来ているわけではなく、以下のような課題を解決して最終的にバイト形式にしています。

- 変数名、関数・メソッド名などの名前解決方法
- リテラルで記述された値の管理・参照方法
- 変数などのメモリエリア確保方法
- 関数・メソッドの中の処理のコンパイル方法
- クラス定義や構造体定義の扱い方

上記では、構文解析やプリプロセッサなど、事前処理は省きました。

コンパイルの過程では、最終的なバイト形式に保存する必要のない一時的に作成する情報もあります。

### 変数名、関数・メソッド名などの名前解決方法

ここからは、C/C++を例に説明します。  
Javaはちょっと違いますが、C/C++よりは簡単に出来ています。

まずC/C++は、プログラムを以下のように分けることが出来ます。

```
main.cpp
util.cpp
```

これらプログラムのコンパイルは以下の手順を踏みます。

1. cppファイルごとにコンパイルしてオブジェクトファイル形式に変換する
2. すべてのオブジェクトファイルを集めて、最終的な実行ファイルにする

上記の 1. にあるように、cppファイル別にコンパイルするため、例えば main.cpp のコンパイルの際には util.cpp にどのような関数があるか分かりません。  
それでも main.cpp から util.cpp の関数を参照したいことがあります。

それを実現するために、関数に対応したラベル名を付けてオブジェクトファイルに保存しておき、別のオブジェクトファイルからはそれを参照できるようにします。

これらラベル名は、最終的な実行ファイルになった時には不要になるため、実行ファイルには保存されません。  
ただし、例えばWindowsのDLLやLinuxのsoファイルなど、共有ライブラリとして実行形式で提供される場合には、実行時にも名前解決をする必要があるため、関数のラベル名が保持されています。

以下から名前解決の例を示します。

```c++
extern "C" void c_func(int a);
void cpp_func(int a);
class MyClass {
public:
    void method(int a);
};
```

コンパイラは、上記のようなメソッドが参照できるように、決められたルールでラベル化します。

```
c_func ⇒ c_func
cpp_func ⇒ _Z8cpp_funci
MyClass::method ⇒ _ZN7MyClassZ6methodEi
```

C言語は、関数名がそのままラベル名として使用されますが、C++は引数が異なるとオーバーロード出来るため、引数も含めた特殊なラベル名となります。  
このラベル名の変換法則を、定義する側と利用する側が同じルールにすることで、cppファイルが別になっても実行ファイルにする時に関連付けることが出来ます。

```c++
extern "C" int c_global;
extern int cpp_global;

class MyClass {
public:
    int field;
    static int s_field;
};
```

また変数名は以下のようになります。

```
c_global ⇒ c_global
cpp_global ⇒ cpp_global（グローバル変数はC言語と変わらない）
field ⇒ 定義されない
s_field ⇒ _ZN7MyClass7s_fieldE
```

グローバル変数はC/C++で変わらず、変数名がそのままラベル名として参照されます。  
クラス変数は、static変数はクラス名が付与されたラベル名となりますが、インスタンス変数に関しては、どこにもラベル名は出てきません。  

```c++
MyClass obj;
obj.field = 3;
```

例えば上記のように field を利用する箇所をコンパイルする際、fieldはインスタンスの最初のメモリにあることが分かるため、変数に対するラベル名を使うのではなく、objが示すアドレスの最初のメモリの値を 3 で書き換えるように変換されるためです。

また、メソッドに戻ります。

```c++
class MyClass {
public:
    virtual void v_method(int a);
};
```

上記のような仮想メソッドは、以下のようなラベルになります。

```
MyClass::v_method ⇒ _ZN7MyClass8v_methodEi
```

ラベル変換ルールは、virtualのついていないメソッドと同じですが、実はこのメソッドを利用する際には使いません。  
この名前は、このメソッドの実体を作った cpp ファイルに対するオブジェクトファイルにのみ存在します。

```c++
MyClacc obj;
obj.v_method(3);
```

上記のような利用する側をコンパイルする際、v_method の名前解決でメソッドを呼び出そうとするのではなく、objのメモリの先頭にある vtable（仮想メソッドテーブル）を参照し、その一番目のインデックスにあるメソッドを呼び出そうとします。  
これは「[関数とクラスの関数](Function.ipynb)」でも説明しましたが、virtualメソッドはオーバーライドされる可能性もあるため、ラベル参照ではなく vtable 経由の呼び出しを行うためです。

### リテラルで記述された値の管理・参照方法

リテラルで記述された値とは、例えば以下のようなものをいいます。

```
const char* MSG = "Hello. World.";

MyClass::MyClass(): SuperClass(3) {
}
```

上記の `"Hello. World."` や `3` のようなプログラムで直接記述している値のことです。

これらの値は実行時にも必要となるものであり、最終的な実行ファイルまで保持されます。  
ただ、この値は各 cpp ファイル内で閉じているため、cppファイル間の参照を考える必要はなく、オブジェクトファイル形式になるときに埋め込まれます。

### 変数などのメモリエリア確保方法

変数は、実行時には必ずメモリ上に領域が確保されます。  
プログラム上では変数名で書いていますが、実行時には変数名は使わず、その変数が示すアドレスに対して読み書きが行われます。  
実行時に、どの変数がどのアドレスにあるかを判断して実行できるよう、コンパイラは最終的なバイト形式にします。

変数が記載できる場所としては以下があります。

```c++
int global;

class MyClass {
public:
    int field;
    static int s_field;
}

void func() {
    int local;
    for (int i = 0; i < 10; i++) {
        int for_local;
    }
    for (int j = 0; j < 10; j++) {
        double for_local2;
    }
}

int MyClass::s_field;  // 変数の実体作成
```

globalとs_fieldは、実行開始時から実行終了まで常に存在する変数であるため、実行ファイルの段階で必要なメモリが確保されるように仕込まれます。  
逆に、関数の中の `local` `i` `for_local` のような変数は、実行中に関数が呼ばれて初めて必要となるメモリエリアです。  
そのため、実行ファイル上に必要なメモリ領域として存在するわけではありません。

ではどうやってメモリが確保されるかというと、一般的なコンパイラでは、この関数のロジックをCPU命令に展開する一番最初に、変数を保持するスタック領域を必要なバイト数分確保します。  
必要なバイト数とは、変数スコープとして同時に存在しうる最大の領域となります。

具体的には、`local` `i` `for_local` が必要となる最初のループ、`local` `j` `for_local2` が必要となる2番目のループを比べて、より領域が必要となる後者のサイズ分となります。  
intの4バイト×2、doubleの8バイトの合計16バイトが関数の最初で確保されるようにCPU命令が書きだされます。

そして、実際にこの変数を使う部分では、`local`であれば0バイト、`i` `j` であれば4バイト、`for_local` `for_local2` であれば8バイトだけ、スタックから移動したメモリ領域を使うようにコンパイルされます。  
（実際は、コンパイルの最適化により順番が変わるかもしれない）  
このように、コンパイルした後は、変数名の情報は一切保持されません。

### 関数・メソッドの中の処理のコンパイル方法

関数やメソッドのコンパイルは、プログラムが動くための肝となる部分です。

```c++
int pow(int a, int b) {
    int ret;
    if (is_zero(b)) {
        return 1;
    }
    ret = a;
    for (int i = 0; i < b; i++) {
        ret *= a;
    }
    return ret;
}
```

あえて別の関数を呼び出す `is_zero` というのを使っています。

上記をコンパイルすると、概ね以下のようなCPU命令に変換されます。

1. 関数の最初で、ローカル変数に必要となる領域 `ret` `i` 用の 8バイトがスタック上で確保される
2. `is_zero(b)`に渡している b の値をメモリから読み込み、メソッド呼び出し用の入力パラメータとなるスタックへプッシュする
3. `is_zero`を呼び出す命令を作成するが、`is_zero`の関数アドレスがまだ分からないため、`is_zero`を参照しているという情報を作成する
4. `is_zero`の戻り値が入っているスタックから情報を取得し、内容を判定する
5. もし内容が 0 だったなら、8. へジャンプする
6. 応答用のスタック領域に戻り値の 1 を書き込む
7. メソッドを抜ける
8. 入力パラメータの a の情報を読み込む
9. `ret`に相当するローカル変数の0バイト目の領域へ読み込んだ a の内容を書きこむ
10. `i`に相当するローカル変数の4バイト目の領域へ 0 を書き込む
11. `i`に相当するローカル変数の4バイト目と、入力パラメータの`b`の内容を比較する（面倒なのでちょっと省略）
12. `i`が`b`以上だったら 18. へジャンプする
13. `ret`に相当するローカル変数の0バイト目の情報を読み込む
14. 読み込んだ情報に入力パラメータの`a`の内容を掛け算する
15. 計算結果を`ret`の領域へ書き込む
16. `i`に相当するローカル変数の4バイト目のメモリの内容を +1 する
17. 11.へジャンプする
18. `ret`に相当するローカル変数0バイト目の内容を、戻り値用のスタック領域へ書き込む
19. メソッドを抜ける

ポイントは、ローカル変数用のエリア確保、CPUが使える簡単な命令へ変換する、他の関数を参照している部分には、未定義ラベルとして後で名前解決できるような情報を付与する、という点です。  
この関数の名前解決について、オブジェクトファイルが合成されて実行ファイルになる際に、この参照先を解決して実際の関数のアドレスへと書き換えます。

ちなみに、`g++ -c xxx.cpp` などでコンパイルして出来るオブジェクトファイル `xxx.o` は、`nm xxx.o` でこのような解決すべきラベルの情報が分かります。  
（説明上はラベルとしましたが、実際はシンボルと呼ぶようです）

### クラス定義や構造体定義の扱い方

クラス定義や構造体の型情報というのは、C/C++ではコンパイルされた後のオブジェクトファイルや実行ファイルには存在しません。  
これらの情報は、コンパイルの際に必要ではありますが、実行時には不要だからです。

```c++
class Base {
protected:
    int field;
    static Base* firstObj;
public:
    Base();
    virtual ~Base();

    void method1();
};

class SubClass: public Base {
public:
    SubClass();
    virtual ~SubClass();
    
    virtual void method2();
    static void s_method();
};
```

例えば上記の情報は、コンパイルの際には以下のように取り扱います。

- Baseクラスの情報
  - virtualメソッドが存在するので vtable が必要
  - fieldの4バイトが必要
  - Baseクラスで必要なメモリサイズは vtable(8バイト)+4バイト+アライメントで 16バイト
  - field変数は vtable ずらした最初のメモリ位置
  - コンストラクタは `_ZN4BaseC1Ev` というラベル名
  - デストラクタは vtable[0] の位置
  - firstObjは `_ZN4Base8firstObjE`というラベル名
  - method1()は`_ZN4Base7method1Ev`というラベル名
- SubClassの情報
  - virtualメソッドがある（なくてもvirtualメソッドがあるBaseを継承している）ため vtable が必要
  - インスタンス変数がないため、Baseと同じく必要なメモリサイズは 16 バイト
  - コンストラクタは `_ZN8SubClassC1Ev` というラベル名
  - デストラクタは vtable[0] の位置
  - method2() は vtable[1] の位置
  - s_method() は `_ZN8SubClass8s_methodEv` というラベル名

オブジェクトファイルの段階では、別ファイル参照のためにラベル名などは保持されますが、最終的な実行ファイルになるときには不要になります。  
また、newする際のメモリサイズも分かっているため、コンパイルの段階で確保すべきメモリサイズとして埋め込まれ、fieldなどの変数へのアクセスも、オブジェクトのメモリ位置から何バイトずらすべきかが分かるため、そのような命令になります。  
メソッド呼び出しも、ラベル参照で最終的に解決されるか、virtualメソッドは vtable のどのインデックスを呼ぶかという命令に変換されます。

このように、クラスの定義情報はコンパイルとしては必要不可欠ですが、実行時にはなくなってしまうものです。

## デバッグ情報

これまでコンパイルするとなくなる情報がいろいろあると書きましたが、それだと実行時デバッグに支障が出ます。  
実行時に break point で止めても、変数名が分からなくなっていたり、どの関数か分からなければデバッグもままなりません。  
そのようなデバッグをサポートする手段として、実行命令ではない補足情報を実行ファイルに付与することが可能です。

```
g++ -g xxx.cpp
```

g++ でいうと、`-g` オプション指定で、デバッグ情報も付与することが出来ます。

デバッグ情報には、本来失われる以下のような情報が入っています。

- 関数名と関数アドレスの対応
- 変数名とメモリ位置の対応情報
- ソースファイルの内容
- CPU命令とソースファイルの行番号との対応

gdbのコマンドは覚えたが、デバッグしてみたらソースファイルが見えないなどは、このデバッグ情報を付与してコンパイルしたかどうかを確認してみてください。